In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# Assuming your CSV data is saved as 'data.csv'
data = pd.read_csv('data_processing/output.csv')

In [21]:
import tensorflow as tf

# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print("GPU:", gpu)
else:
    print("No GPU available, using CPU.")


GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [22]:
# Preprocessing
# One-hot encoding for categorical variables
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(data[['curr_attack', 'interrupted']])
encoded_labels = pd.get_dummies(data['mtd'])

# Combine one-hot encoded features with other numerical features
features = pd.concat([pd.DataFrame(encoded_features), data[['mtd_freq', 'compromised_num']]], axis=1)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

In [23]:
X_train, y_train

(       0    1    2    3    4    5    6    7  mtd_freq  compromised_num
 79   0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005030              120
 217  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005001              383
 113  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005015              173
 140  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.005004              243
 122  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.005004              192
 ..   ...  ...  ...  ...  ...  ...  ...  ...       ...              ...
 106  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005019              161
 14   0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.005129                5
 92   0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.005024              144
 179  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005001              316
 102  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005008              158
 
 [176 rows x 10 columns],
      CompleteTopologyShuffle  IPShuffle  OSDiversity  ServiceDiversity
 79                     False       

In [24]:
# Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # Output layer: 4 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


/Users/williamho/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.2690 - loss: 9.1020 - val_accuracy: 0.3611 - val_loss: 2.0131
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2282 - loss: 2.3634 - val_accuracy: 0.3056 - val_loss: 2.2342
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4370 - loss: 2.1355 - val_accuracy: 0.4167 - val_loss: 1.6175
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3938 - loss: 1.6005 - val_accuracy: 0.3889 - val_loss: 1.7125
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5210 - loss: 1.5026 - val_accuracy: 0.4167 - val_loss: 1.5997
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4141 - loss: 1.5059 - val_accuracy: 0.2778 - val_loss: 1.6481
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4268 - loss: 1.4776 - val_accuracy: 0.6667 - val_loss: 1.2384
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3919 - loss: 1.3469 - val_accuracy: 0.

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
model.evaluate(X_test, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5871 - loss: 0.7589


[0.7823486328125, 0.5681818127632141]

In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense


data = pd.read_csv('data_processing/testing_dataset.csv')


X = data.drop('name', axis = 1)
y = data['name']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))  # Output layer with 4 units for 4 classes and softmax activation

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=10, validation_split=0.2)




Epoch 1/20


/Users/williamho/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


480/480 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.4000 - loss: 1.5653 - val_accuracy: 0.5200 - val_loss: 0.7815
Epoch 2/20
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5750 - loss: 0.8292 - val_accuracy: 0.5200 - val_loss: 0.6771
Epoch 3/20
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5900 - loss: 0.7281 - val_accuracy: 0.5950 - val_loss: 0.6434
Epoch 4/20
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5881 - loss: 0.7315 - val_accuracy: 0.5200 - val_loss: 0.8098
Epoch 5/20
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5960 - loss: 0.6910 - val_accuracy: 0.5200 - val_loss: 0.6745
Epoch 6/20
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.6174 - loss: 0.6591 - val_accuracy: 0.5200 - val_loss: 0.6439
Epoch 7/20
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.6388 - loss: 0.6503 - val_accuracy: 0.5200 - val_loss: 0.6366
Epoch 8/20
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.6445 - loss: 0.6317 - val_accuracy: 0.520

In [27]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.1096
Test loss: 0.11629361659288406, Test accuracy: 1.0
